In [12]:
!pip install altair

In [13]:
import pandas as pd
import numpy as np
import altair as alt
from matplotlib import pyplot as plt
from matplotlib.dates import date2num, num2date
from matplotlib import dates as mdates
from matplotlib import ticker
from matplotlib.colors import ListedColormap
from matplotlib.patches import Patch
import seaborn as sns
import math
from scipy import stats as sps
from scipy.interpolate import interp1d

from IPython.display import clear_output

In [14]:
#Load and preprocess data

def get_process_data(url = 'https://health-infobase.canada.ca/src/data/covidLive/covid19-download.csv', 
                     regions = ['British Columbia','Alberta','Saskatchewan','Manitoba','Ontario','Quebec']):
  """Loads data from source to dataframe & preprocess it"""
  # Select provinces with population > 1 million
  data = pd.read_csv(url,
                     usecols=['date', 'prname', 'numtotal'],
                     parse_dates=['date'],
                     index_col=['prname', 'date'],
                     squeeze=True).sort_index()



  idx = pd.IndexSlice
  data = data.loc[idx[regions,:]]
  return data


In [15]:
data = get_process_data()

In [16]:
#Smoothing the time series data (New Daily Cases) by applying Gaussian Filter

def smooth_cases(cases, cutoff = 20):
  new_cases = cases.diff()

  smoothed = new_cases.rolling(7, win_type = 'gaussian', min_periods = 1, center = True).mean(std = 2).round()
  idx_start = np.searchsorted(smoothed, cutoff)
  smoothed = smoothed.iloc[idx_start:]
  original = new_cases.loc[smoothed.index]

  return original, smoothed

In [17]:
#Calculating posterior probabilities for Reproduction Number
def get_posteriors(new_cases_smoothed, sigma = 0.15, gam = 1/7, r_t_max = 12):
  '''Function to compute posterior probabilities. Input variables are sigma, gamma blah________________________________?????????????????'''
  r_t_range = np.linspace(0, r_t_max, r_t_max*100+1)
  # Calculate Lambda
  lam = new_cases_smoothed[:-1].values * np.exp(gam * (r_t_range[:, None] - 1)) 
  #print('lam: ',lam.shape)

  #Calculate each days likelihood
  likelihoods = pd.DataFrame(data = sps.poisson.pmf(new_cases_smoothed[1:], lam),
                             index = r_t_range,
                             columns = new_cases_smoothed.index[1:])
  #print('likelihoods: ',likelihoods.shape)
  
  # Create Gaussian and normalize
  process_matrix = sps.norm(loc=r_t_range,
                              scale=sigma
                             ).pdf(r_t_range[:, None]) 

  process_matrix /= process_matrix.sum(axis=0)

  # (4) Calculate the initial prior
    #prior0 = sps.gamma(a=4).pdf(r_t_range)
  prior0 = np.ones_like(r_t_range)/len(r_t_range)
  prior0 /= prior0.sum()

  # Create a DataFrame that will hold our posteriors for each day
    # Insert our prior as the first posterior.
  posteriors = pd.DataFrame(
      index=r_t_range,
      columns=new_cases_smoothed.index,
      data={new_cases_smoothed.index[0]: prior0}   
    )   

  # We said we'd keep track of the sum of the log of the probability
    # of the data for maximum likelihood calculation.
  log_likelihood = 0.0

    # (5) Iteratively apply Bayes' rule
  for previous_day, current_day in zip(new_cases_smoothed.index[:-1], new_cases_smoothed.index[1:]):

        #(5a) Calculate the new prior
      current_prior = process_matrix @ posteriors[previous_day]
        
        #(5b) Calculate the numerator of Bayes' Rule: P(k|R_t)P(R_t)
      numerator = likelihoods[current_day] * current_prior
        
        #(5c) Calcluate the denominator of Bayes' Rule P(k)
      denominator = np.sum(numerator)
        
        # Execute full Bayes' Rule
      posteriors[current_day] = numerator/denominator
        
        # Add to the running sum of log likelihoods
      log_likelihood += np.log(denominator)
    
  return posteriors, log_likelihood




In [18]:
def highest_density_interval(pmf, p=.9, debug=False):
    # If we pass a DataFrame, just call this recursively on the columns
    if(isinstance(pmf, pd.DataFrame)):
        return pd.DataFrame([highest_density_interval(pmf[col], p=p) for col in pmf],
                            index=pmf.columns)
    
    cumsum = np.cumsum(pmf.values)
    
    # N x N matrix of total probability mass for each low, high
    total_p = cumsum - cumsum[:, None]
    
    # Return all indices with total_p > p
    lows, highs = (total_p > p).nonzero()
    
    # Find the smallest range (highest density)
    best = (highs - lows).argmin()
    
    low = pmf.index[lows[best]]
    high = pmf.index[highs[best]]
    
    return pd.Series([low, high],
                     index=[f'Low_{p*100:.0f}',
                            f'High_{p*100:.0f}'])




In [8]:
def calc_results(data):

  sigmas = np.linspace(1/20, 1, 20)
  results = {}
  for region_name, cases in data.groupby(level='prname'):
    print('Calculating, this will take a few minutes- thanks for waiting :-)')
    print(region_name)
    new, smoothed = smooth_cases(cases, cutoff=10)
    
    if len(smoothed) == 0:
        new, smoothed = smooth_cases(cases, cutoff=10)
    
    result = {}
    
    # Holds all posteriors with every given value of sigma
    result['posteriors'] = []
    
    # Holds the log likelihood across all k for each value of sigma
    result['log_likelihoods'] = []
    
    for sigma in sigmas:
      posteriors, log_likelihood = get_posteriors(smoothed, sigma=sigma)
      result['posteriors'].append(posteriors)
      result['log_likelihoods'].append(log_likelihood)
    
    # Store all results keyed off of state name
    results[region_name] = result
    clear_output(wait=True)

  # Each index of this array holds the total of the log likelihoods for
  # the corresponding index of the sigmas array.
  total_log_likelihoods = np.zeros_like(sigmas)

  # Loop through each state's results and add the log likelihoods to the running total.
  for state_name, result in results.items():
    total_log_likelihoods += result['log_likelihoods']

  # Select the index with the largest log likelihood total
  max_likelihood_index = total_log_likelihoods.argmax()

  # Select the value that has the highest log likelihood
  sigma = sigmas[max_likelihood_index]

  final_results = None

  for region_name, result in results.items():
    print('Calculating, this will take a few minutes- thanks for waiting :-)')
    print(region_name)
    posteriors = result['posteriors'][max_likelihood_index]
    hdis_90 = highest_density_interval(posteriors, p=.9)
    hdis_50 = highest_density_interval(posteriors, p=.5)
    most_likely = posteriors.idxmax().rename('Rt')
    result = pd.concat([most_likely, hdis_90, hdis_50], axis=1)
    if final_results is None:
        final_results = result
    else:
        final_results = pd.concat([final_results, result])
    clear_output(wait=True)

  print('Done.')

  return sigma, results, final_results

sigma, results, final_results = calc_results(data)

Done.


In [9]:



def plot_rt(data,region):
  idx = pd.IndexSlice
  source = data.loc[idx[[region],:]].reset_index(level = 'date')

  base = alt.Chart(source, title = 'Time Series of Rt for Canadian Provinces').encode(x = alt.X('date',title = ''),
                                            y = alt.Y('Rt', title = 'Rt'),
                                            tooltip = ['date','Rt']).interactive()


  line = alt.Chart(pd.DataFrame({'y': [1]})).mark_rule(color="red", opacity=0.35, size=1).encode(y='y')

  band = alt.Chart(source).mark_area(opacity=0.5, color='gray').encode(x='date',y= alt.Y('Low_90', title=''),y2=alt.Y2('High_90',title=''))

  display(base+ line + band)



In [10]:
#Final Plotting
regions = ['Ontario','Quebec','British Columbia', 'Alberta','Saskatchewan','Manitoba']
df = final_results.reset_index()
for region in regions:
  df[df.prname == region] = df[df.prname == region][1:]
df.dropna(inplace = True)
df.head()


,prname,date,Rt,Low_90,High_90,Low_50,High_50
1,Alberta,2020-03-16,4.05,1.08,6.47,2.77,5.03
2,Alberta,2020-03-17,3.49,1.52,5.17,2.64,4.14
3,Alberta,2020-03-18,3.11,1.60,4.40,2.44,3.59
4,Alberta,2020-03-19,2.75,1.51,3.82,2.19,3.14
5,Alberta,2020-03-20,2.41,1.35,3.35,1.95,2.77


In [23]:
base = alt.Chart(df).mark_line(color = 'teal').encode(x = alt.X('date',title = ''),
                                            y = alt.Y('Rt', title = 'Rt'),
                                            tooltip = ['date','Rt','Low_90','High_90']).interactive()


band = alt.Chart(df).mark_area(opacity=0.4, color='gray').encode(x='date',y= alt.Y('Low_90', title=''),y2=alt.Y2('High_90',title=''))

alt.layer(base, band, data=df).facet('prname',columns = 3).resolve_axis(
    x='independent',
    y='independent',
).configure(background='#fdfefe')

alt.FacetChart(...)